In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re
import cloudscraper
import random

headers = {
    'User-Agent':  "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"
}

# Low 100M - 500M

In [2]:
base_url = 'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=500000000&minPrice=100000000&page=1'

In [3]:
response = requests.get(base_url, headers=headers)

In [4]:
response.status_code

200

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
max_page = soup.find_all('li', class_='ui-molecule-paginate__item')

In [7]:
last_index = max_page[len(max_page)-1]

In [8]:
last_index

<li class="ui-molecule-paginate__item"><a aria-label="Page 75" href="https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=500000000&amp;minPrice=100000000&amp;page=75" tabindex="0">75</a></li>

In [9]:
last_index.text

'75'

In [10]:
scraper = cloudscraper.create_scraper()

In [11]:
data = []

In [12]:
for i in range(1, 40):
    url = f'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=500000000&minPrice=100000000&page={i}'
    base_url = 'https://www.rumah123.com'

    response = scraper.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(random.uniform(3, 5))

    ad_divs = soup.find_all('div', class_=re.compile('^ui-organism-intersection__element intersection-card-container'))

    print(f'total ad: {len(ad_divs)}')
    
    ad_links = []
    for a_tag in ad_divs:
        if a_tag.find('a', href=re.compile('^/perumahan-baru/properti/')):
            continue
        try:    
            link = a_tag.find('a', href=re.compile('^/properti/surabaya/'))
            link = base_url + link['href']

            sub_response = scraper.get(link, headers=headers)
            sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
            time.sleep(2)

            sub_data = {}
            print(f'ITER: {i}')

            harga_nama_div = sub_soup.find('div', class_='border-0 border-b border-solid border-gray-200 mb-4 mt-3')
            harga = harga_nama_div.find('span').text
            nama = harga_nama_div.find('h1').text
            print(f'{nama} --- {harga}')
            sub_data['Nama'] = nama
            sub_data['Harga'] = harga
        
            properti_div = sub_soup.find('div', id='property-information')
            info_prop = properti_div.find_all('p', class_='w-32 text-xs font-light text-gray-500')
        
            for prop in info_prop:
                key = prop.text
                val = prop.next_sibling.text
                
                print(f'{key}: {val}')
                sub_data[key] = val
            print("")

            data.append(sub_data)
        except:
            continue
        
    print("="*100)

total ad: 20
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
total ad: 20
ITER: 2
ITER: 2
ITER: 2
Dijual Aset Lelang ,tambak Mayor Baru Timur ,asemrowo Surabaya --- Rp 150 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 120 m²
Luas Bangunan: 100 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Garasi: 1
Jumlah Lantai: 1
Kondisi Properti: Renovasi Minimum

ITER: 2
Dijual Via Lelang Rumah Bagus Lokasi Simomulyo Tandes --- Rp 254 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 120 m²
Luas Bangunan: 90 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 1
Kondisi Properti: Bagus

ITER: 2
RUMAH WISMA TENGGER PIPA SURABAYA 83499 --- Rp 482 Juta
Kamar Tidur: 1
Kamar Mandi: 1
Luas Tanah: 120 m²
Luas Bangunan: 10 m²
Sertifikat: SHM
Daya Listrik: Lainnya
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Jumlah Lantai: 1
Kondisi Properti: Bagus

ITER: 2
Dijual Via Lelang R

In [13]:
len(data)

237

In [14]:
df_low = pd.DataFrame(data)
df_low = df_low.fillna(0)
df_low['Category'] = 'Low'
df_low.to_csv('Surabaya_low.csv', index=False)

In [15]:
df_low

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Garasi,Jumlah Lantai,Kondisi Properti,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,"Dijual Aset Lelang ,tambak Mayor Baru Timur ,a...",Rp 150 Juta,2,1,120 m²,100 m²,1,SHM,Lainnya,1,1,Renovasi Minimum,0,0,Low
1,Dijual Via Lelang Rumah Bagus Lokasi Simomulyo...,Rp 254 Juta,2,1,120 m²,90 m²,1,SHM,Lainnya,0,1,Bagus,0,0,Low
2,RUMAH WISMA TENGGER PIPA SURABAYA 83499,Rp 482 Juta,1,1,120 m²,10 m²,0,SHM,Lainnya,0,1,Bagus,1,1,Low
3,Dijual Via Lelang Rmh 1 Lt Di Tandes Surabaya,Rp 254 Juta,3,1,120 m²,90 m²,1,SHM,Lainnya,1,1,Bagus,0,0,Low
4,Dijual Rumah Murah Lokasi Benowo,Rp 335 Juta,2,1,24 m²,19 m²,0,SHM,Lainnya,1,1,Baru,0,0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Perumahan Nara Park Surabaya Timur - Hunian St...,Rp 390 Juta,2,1,41 m²,40 m²,0,Lainnya,Lainnya Watt,0,0,0,0,0,Low
233,Rumah Baru Vicadha Residence Terletak di Jalur...,Rp 166 Juta,2,1,72 m²,30 m²,1,HGB,1300 Watt,1,1,Baru,0,0,Low
234,Dijual Rumah Baru Gress. Kinyis² \nada 2 Unit ...,Rp 450 Juta,2,1,50 m²,50 m²,1,SHM,1300 Watt,1,2,Baru,0,0,Low
235,APARTEMEN TANGLIN FULL FURNISH,Rp 400 Juta,1,1,22 m²,22 m²,0,Lainnya,2200 Watt,0,0,0,0,0,Low


### Low-Mid 500M-1B

In [10]:
scraper = cloudscraper.create_scraper()

In [11]:
data_low_mid = []

In [12]:
for i in range(1, 40):
    url = f'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=1000000000&minPrice=500000000&page={i}'
    base_url = 'https://www.rumah123.com'

    response = scraper.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(random.uniform(3, 5))

    ad_divs = soup.find_all('div', class_=re.compile('^ui-organism-intersection__element intersection-card-container'))

    print(f'total ad: {len(ad_divs)}')
    
    ad_links = []
    for a_tag in ad_divs:
        if a_tag.find('a', href=re.compile('^/perumahan-baru/properti/')):
            continue
        try:    
            link = a_tag.find('a', href=re.compile('^/properti/surabaya/'))
            link = base_url + link['href']

            sub_response = scraper.get(link, headers=headers)
            sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
            time.sleep(random.uniform(3, 5))

            sub_data = {}
            print(f'ITER: {i}')

            harga_nama_div = sub_soup.find('div', class_='border-0 border-b border-solid border-gray-200 mb-4 mt-3')
            harga = harga_nama_div.find('span').text
            nama = harga_nama_div.find('h1').text
            print(f'{nama} --- {harga}')
            sub_data['Nama'] = nama
            sub_data['Harga'] = harga
        
            properti_div = sub_soup.find('div', id='property-information')
            info_prop = properti_div.find_all('p', class_='w-32 text-xs font-light text-gray-500')
        
            for prop in info_prop:
                key = prop.text
                val = prop.next_sibling.text
                
                print(f'{key}: {val}')
                sub_data[key] = val
            print("")

            data_low_mid.append(sub_data)
        except:
            continue
        
    print("="*100)

total ad: 21
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
ITER: 1
Pakal Madya Indah Sby Mepet Citraland Utr Paling.murah ! --- Rp 775 Juta
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 84 m²
Luas Bangunan: 56 m²
Carport: 1
Sertifikat: HGB
Daya Listrik: 2200 Watt
Jumlah Lantai: 1
Kondisi Properti: Bagus

ITER: 1
Dijual Rumah 1 Lantai Lokasi Bulak Kali Tinjang Kenjeran --- Rp 790 Juta
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 85 m²
Luas Bangunan: 70 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 1
Kondisi Properti: Bagus

ITER: 1
Rumah Jual Murah Butuh Uang Sukolilo Dian Regency  ---  
Kamar Tidur: 2
Kamar Mandi: 1
Luas Tanah: 90 m²
Luas Bangunan: 80 m²
Carport: 1
Sertifikat: HGB
Daya Listrik: Lainnya
Jumlah Lantai: 1
Kondisi Properti: Bagus

ITER: 1
Rumah Mewah 2 Lantai Bisa Untuk Usaha Hook Jalan Raya Wiguna --- Rp 870 Juta
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 56 m²
Luas Bangunan: 80 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 1300 Watt
Garasi: 2
Jumlah 

In [13]:
len(data_low_mid)

263

In [14]:
df_low_mid = pd.DataFrame(data_low_mid)
df_low_mid = df_low_mid.fillna(0)
df_low_mid['Category'] = 'Low-Mid'
df_low_mid.to_csv('Surabaya_low_mid.csv', index=False)

In [15]:
df_low_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Garasi,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,Pakal Madya Indah Sby Mepet Citraland Utr Pali...,Rp 775 Juta,2,1,84 m²,56 m²,1,HGB,2200 Watt,1,Bagus,0,0,0,Low-Mid
1,Dijual Rumah 1 Lantai Lokasi Bulak Kali Tinjan...,Rp 790 Juta,3,2,85 m²,70 m²,2,SHM,Lainnya,1,Bagus,0,0,0,Low-Mid
2,Rumah Jual Murah Butuh Uang Sukolilo Dian Rege...,,2,1,90 m²,80 m²,1,HGB,Lainnya,1,Bagus,0,0,0,Low-Mid
3,Rumah Mewah 2 Lantai Bisa Untuk Usaha Hook Jal...,Rp 870 Juta,3,2,56 m²,80 m²,2,SHM,1300 Watt,2,Baru,2,0,0,Low-Mid
4,Rumah Mewah Wonorejo Selatan Rungkut,,2,1,100 m²,70 m²,0,SHM,Lainnya,1,Bagus,0,0,0,Low-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,Baru Penjaringan Dkt Rungkut Medokan Medayu Wi...,Rp 850 Juta,3,2,45 m²,80 m²,1,SHM,Lainnya,2,Bagus,0,0,0,Low-Mid
259,Baru Medayu Utara Dkt Baruk Rungkut Wiguna Pan...,Rp 750 Juta,3,1,100 m²,55 m²,1,SHM,Lainnya,1,Bagus,0,0,0,Low-Mid
260,Baru Pondok Benowo Dkt Citraland Pakuwon Semem...,Rp 758 Juta,3,2,78 m²,110 m²,1,SHM,Lainnya,2,Bagus,0,0,0,Low-Mid
261,Rumah Dp 0% Dekat Gunung Anyar Surabaya Timur ...,Rp 600 Juta,2,1,90 m²,36 m²,1,SHM,Lainnya,1,Baru,0,0,0,Low-Mid


### Up-Mid 1B-3B

In [16]:
scraper = cloudscraper.create_scraper()

In [17]:
data_up_mid = []

In [18]:
for i in range(1, 40):
    url = f'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=3000000000&minPrice=1000000000&page={i}'
    base_url = 'https://www.rumah123.com'

    response = scraper.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(random.uniform(3, 5))

    ad_divs = soup.find_all('div', class_=re.compile('^ui-organism-intersection__element intersection-card-container'))

    print(f'total ad: {len(ad_divs)}')
    
    ad_links = []
    for a_tag in ad_divs:
        if a_tag.find('a', href=re.compile('^/perumahan-baru/properti/')):
            continue
        try:    
            link = a_tag.find('a', href=re.compile('^/properti/surabaya/'))
            link = base_url + link['href']

            sub_response = scraper.get(link, headers=headers)
            sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
            time.sleep(random.uniform(3, 5))

            sub_data = {}
            print(f'ITER: {i}')

            harga_nama_div = sub_soup.find('div', class_='border-0 border-b border-solid border-gray-200 mb-4 mt-3')
            harga = harga_nama_div.find('span').text
            nama = harga_nama_div.find('h1').text
            print(f'{nama} --- {harga}')
            sub_data['Nama'] = nama
            sub_data['Harga'] = harga
        
            properti_div = sub_soup.find('div', id='property-information')
            info_prop = properti_div.find_all('p', class_='w-32 text-xs font-light text-gray-500')
        
            for prop in info_prop:
                key = prop.text
                val = prop.next_sibling.text
                
                print(f'{key}: {val}')
                sub_data[key] = val
            print("")

            data_up_mid.append(sub_data)
        except:
            continue
        
    print("="*100)

total ad: 24
ITER: 1
SHM Rumah Baru Mewah Area Merr Rungkut Mewah Free Dp --- Rp 1,5 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 75 m²
Luas Bangunan: 100 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2
Kondisi Properti: Baru

ITER: 1
Dijual Rumah 2 Lantai Ready Unit Baru Lokasi Lakarsantri Dekat Pakuwon ---  
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 60 m²
Luas Bangunan: 90 m²
Sertifikat: SHM
Daya Listrik: 2200 Watt
Garasi: 2
Jumlah Lantai: 2
Kondisi Properti: Baru

ITER: 1
Dijual Rumah Bagus 2 Lantai Full Furnished Lokasi Grand Island Pakuwon City --- Rp 2,3 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 72 m²
Luas Bangunan: 80 m²
Carport: 1
Sertifikat: Lainnya
Daya Listrik: Lainnya
Jumlah Lantai: 2
Kondisi Properti: Bagus

ITER: 1
Rumah Ready Unit 2 Lantai Raya Prapen Surabaya Selatan --- Rp 1,51 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 50 m²
Luas Bangunan: 61 m²
Carport: 1
Sertifikat: SHM
Daya Listrik: 2200 Watt
Jumlah Lantai: 2
Kondisi Properti

In [20]:
len(data_up_mid)

284

In [19]:
df_up_mid = pd.DataFrame(data_up_mid)
df_up_mid = df_up_mid.fillna(0)
df_up_mid['Category'] = 'Up-Mid'
df_up_mid.to_csv('Surabaya_up_mid.csv', index=False)

In [21]:
df_up_mid

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Garasi,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Category
0,SHM Rumah Baru Mewah Area Merr Rungkut Mewah F...,"Rp 1,5 Miliar",3,2,75 m²,100 m²,1,SHM,2200 Watt,2,Baru,0,0,0,Up-Mid
1,Dijual Rumah 2 Lantai Ready Unit Baru Lokasi L...,,3,2,60 m²,90 m²,0,SHM,2200 Watt,2,Baru,2,0,0,Up-Mid
2,Dijual Rumah Bagus 2 Lantai Full Furnished Lok...,"Rp 2,3 Miliar",3,2,72 m²,80 m²,1,Lainnya,Lainnya,2,Bagus,0,0,0,Up-Mid
3,Rumah Ready Unit 2 Lantai Raya Prapen Surabaya...,"Rp 1,51 Miliar",3,2,50 m²,61 m²,1,SHM,2200 Watt,2,Baru,0,0,0,Up-Mid
4,Dijual Rumah Baru 2 Lantai Lokasi Manyar Tirto...,"Rp 2,55 Miliar",3,3,100 m²,175 m²,1,SHM,4400 Watt,2,Bagus,0,1,1,Up-Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Jual Rumah Baru Mungil Dan Elok Full Bangunan ...,"Rp 1,71 Miliar",3,3,100 m²,138 m²,1,SHM,2200 Watt,2,Baru,0,1,1,Up-Mid
280,Dijual Rumah New The Juanda Sidoarjo Fasilitas...,"Rp 1,1 Miliar",2,2,74 m²,72 m²,1,SHM,1300 Watt,2,Bagus,0,0,0,Up-Mid
281,Jual Rumah Northwest Lake 2 Bedroom View Taman,"Rp 1,3 Miliar",2,2,55 m²,55 m²,0,HGB,Lainnya,2,Bagus,0,0,0,Up-Mid
282,Jual Rumah Rungkut Harapan,"Rp 2,3 Miliar",3,3,205 m²,130 m²,0,SHM,2200 Watt,1,Bagus,0,1,1,Up-Mid


### High 3B-7B

In [25]:
scraper = cloudscraper.create_scraper()

In [26]:
data_high = []

In [27]:
for i in range(1, 40):
    url = f'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=7000000000&minPrice=3000000000&page={i}'
    base_url = 'https://www.rumah123.com'

    response = scraper.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(random.uniform(3, 5))

    ad_divs = soup.find_all('div', class_=re.compile('^ui-organism-intersection__element intersection-card-container'))

    print(f'total ad: {len(ad_divs)}')
    
    ad_links = []
    for a_tag in ad_divs:
        if a_tag.find('a', href=re.compile('^/perumahan-baru/properti/')):
            continue
        try:    
            link = a_tag.find('a', href=re.compile('^/properti/surabaya/'))
            link = base_url + link['href']

            sub_response = scraper.get(link, headers=headers)
            sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
            time.sleep(random.uniform(3, 5))

            sub_data = {}
            print(f'ITER: {i}')

            harga_nama_div = sub_soup.find('div', class_='border-0 border-b border-solid border-gray-200 mb-4 mt-3')
            harga = harga_nama_div.find('span').text
            nama = harga_nama_div.find('h1').text
            print(f'{nama} --- {harga}')
            sub_data['Nama'] = nama
            sub_data['Harga'] = harga
        
            properti_div = sub_soup.find('div', id='property-information')
            info_prop = properti_div.find_all('p', class_='w-32 text-xs font-light text-gray-500')
        
            for prop in info_prop:
                key = prop.text
                val = prop.next_sibling.text
                
                print(f'{key}: {val}')
                sub_data[key] = val
            print("")

            data_high.append(sub_data)
        except:
            continue
        
    print("="*100)

total ad: 24
ITER: 1
Dijual Setengah Dari Harga Pasar Via Lelang Rumah Jl. Residen Sudirman Surabaya --- Rp 4,5 Miliar
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 372 m²
Luas Bangunan: 250 m²
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lantai: 1
Kondisi Properti: Renovasi Total

ITER: 1
RUMAH MEWAH BARU GRESS GRAHA FAMILY SIAP HUNI. Hadap Barat Daya. Row Jalan 3 MOBIL --- Rp 5,5 Miliar
Kamar Tidur: 5
Kamar Mandi: 4
Luas Tanah: 210 m²
Luas Bangunan: 350 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 4400 Watt
Kamar Tidur Pembantu: 2
Kamar Mandi Pembantu: 1
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

ITER: 1
Dekat Galaxy Mall Rumah Kertajaya Indah Timur Turun Harga Shm ---  
Kamar Tidur: 3
Kamar Mandi: 2
Luas Tanah: 300 m²
Luas Bangunan: 200 m²
Sertifikat: SHM
Daya Listrik: 4400 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 1
Jumlah Lantai: 2
Kondisi Properti: Bagus

ITER: 1
Cluster Luxury Grand Palais.satu2nya Cluster Di Wisata Bukit Mas Dengan Vibes Pakuwonbaru Renov

In [29]:
len(data_high)

269

In [28]:
df_high = pd.DataFrame(data_high)
df_high = df_high.fillna(0)
df_high['Category'] = 'High'
df_high.to_csv('Surabaya_high.csv', index=False)

In [30]:
df_high

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Sertifikat,Daya Listrik,Jumlah Lantai,Kondisi Properti,Carport,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Category
0,Dijual Setengah Dari Harga Pasar Via Lelang Ru...,"Rp 4,5 Miliar",3,2,372 m²,250 m²,SHM,Lainnya,1,Renovasi Total,0,0,0,0,High
1,RUMAH MEWAH BARU GRESS GRAHA FAMILY SIAP HUNI....,"Rp 5,5 Miliar",5,4,210 m²,350 m²,SHM,4400 Watt,2,Bagus,2,2,1,1,High
2,Dekat Galaxy Mall Rumah Kertajaya Indah Timur ...,,3,2,300 m²,200 m²,SHM,4400 Watt,2,Bagus,0,1,1,1,High
3,Cluster Luxury Grand Palais.satu2nya Cluster D...,,4,3,180 m²,230 m²,SHM,4400 Watt,2,Sudah Renovasi,2,1,1,0,High
4,"Rumah Usaha Raya Tidar, Shm, Bisa Saya Bantu Kpr","Rp 6,5 Miliar",0,2,130 m²,140 m²,SHM,5500 Watt,2,Bagus,2,0,0,0,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,darmo baru barat row jalan lebar,"Rp 3,95 Miliar",7,4,240 m²,310 m²,SHM,2200 Watt,1,Butuh Renovasi,0,0,0,0,High
265,Rumah Wisata Bukit Mas Dulovre Surabaya Murah ...,"Rp 3,75 Miliar",4,2,180 m²,200 m²,SHM,2200 Watt,2,Bagus,0,1,1,0,High
266,Dijual Rumah Wisata Bukit Mas Dekat Royal Resi...,"Rp 3,15 Miliar",3,3,180 m²,250 m²,SHM,3500 Watt,2,Bagus,2,0,0,0,High
267,Rumah Kos Bumi Marina Emas Utara Cocok Untuk U...,Rp 3 Miliar,11,10,240 m²,240 m²,SHM,5500 Watt,2,Bagus,1,2,1,0,High


### Luxury 7B-20B

In [33]:
scraper = cloudscraper.create_scraper()

In [34]:
data_luxury = []

In [35]:
for i in range(1, 40):
    url = f'https://www.rumah123.com/jual/surabaya/rumah/?maxPrice=20000000000&minPrice=7000000000&page={i}'
    base_url = 'https://www.rumah123.com'

    response = scraper.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    time.sleep(random.uniform(3, 5))

    ad_divs = soup.find_all('div', class_=re.compile('^ui-organism-intersection__element intersection-card-container'))

    print(f'total ad: {len(ad_divs)}')
    
    ad_links = []
    for a_tag in ad_divs:
        if a_tag.find('a', href=re.compile('^/perumahan-baru/properti/')):
            continue
        try:    
            link = a_tag.find('a', href=re.compile('^/properti/surabaya/'))
            link = base_url + link['href']

            sub_response = scraper.get(link, headers=headers)
            sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
            time.sleep(random.uniform(3, 5))

            sub_data = {}
            print(f'ITER: {i}')

            harga_nama_div = sub_soup.find('div', class_='border-0 border-b border-solid border-gray-200 mb-4 mt-3')
            harga = harga_nama_div.find('span').text
            nama = harga_nama_div.find('h1').text
            print(f'{nama} --- {harga}')
            sub_data['Nama'] = nama
            sub_data['Harga'] = harga
        
            properti_div = sub_soup.find('div', id='property-information')
            info_prop = properti_div.find_all('p', class_='w-32 text-xs font-light text-gray-500')
        
            for prop in info_prop:
                key = prop.text
                val = prop.next_sibling.text
                
                print(f'{key}: {val}')
                sub_data[key] = val
            print("")

            data_luxury.append(sub_data)
        except:
            continue
        
    print("="*100)

total ad: 20
ITER: 1
ITER: 1
Dijual Rumah Mewah Margorejo Indah Semi Furnished --- Rp 9,2 Miliar
Kamar Tidur: 5
Kamar Mandi: 3
Luas Tanah: 400 m²
Luas Bangunan: 500 m²
Carport: 2
Sertifikat: SHM
Daya Listrik: 10600 Watt
Kamar Tidur Pembantu: 1
Kamar Mandi Pembantu: 1
Garasi: 2
Jumlah Lantai: 2
Kondisi Properti: Bagus

ITER: 1
Dijual Rumah Tengah Kota Surabaya Cocok Unt Kos Dan Cafe --- Rp 12 Miliar
Kamar Tidur: 22
Kamar Mandi: 8
Luas Tanah: 365 m²
Luas Bangunan: 534 m²
Sertifikat: SHM
Daya Listrik: 5500 Watt
Garasi: 3
Kondisi Properti: Bagus

ITER: 1
Jual Cepat Rumah Hitung Tanah Shm Di Jl. Embong Wungu Pusat Kota --- Rp 18 Miliar
Luas Tanah: 494 m²
Luas Bangunan: 450 m²
Sertifikat: SHM
Daya Listrik: 4400 Watt
Jumlah Lantai: 1
Kondisi Properti: Renovasi Total

ITER: 1
Rumah Mewah Bangunan Baru Gress selangkah Ke raya Darmo dekat Mayjend Surabaya Pusat --- Rp 18 Miliar
Kamar Tidur: 6
Kamar Mandi: 8
Luas Tanah: 580 m²
Luas Bangunan: 750 m²
Sertifikat: SHM
Daya Listrik: Lainnya
Jumlah Lan

In [37]:
len(data_luxury)

285

In [38]:
df_luxury = pd.DataFrame(data_luxury)
df_luxury = df_luxury.fillna(0)
df_luxury['Category'] = 'Luxury'
df_luxury.to_csv('Surabaya_luxury.csv', index=False)

In [39]:
df_luxury

,Nama,Harga,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Sertifikat,Daya Listrik,Kamar Tidur Pembantu,Kamar Mandi Pembantu,Garasi,Jumlah Lantai,Kondisi Properti,Category
0,Dijual Rumah Mewah Margorejo Indah Semi Furnished,"Rp 9,2 Miliar",5,3,400 m²,500 m²,2,SHM,10600 Watt,1,1,2,2,Bagus,Luxury
1,Dijual Rumah Tengah Kota Surabaya Cocok Unt Ko...,Rp 12 Miliar,22,8,365 m²,534 m²,0,SHM,5500 Watt,0,0,3,0,Bagus,Luxury
2,Jual Cepat Rumah Hitung Tanah Shm Di Jl. Embon...,Rp 18 Miliar,0,0,494 m²,450 m²,0,SHM,4400 Watt,0,0,0,1,Renovasi Total,Luxury
3,Rumah Mewah Bangunan Baru Gress selangkah Ke r...,Rp 18 Miliar,6,8,580 m²,750 m²,0,SHM,Lainnya,0,0,0,2,Bagus,Luxury
4,Rumah Citraland Minimalis Modern Dekat Kampus ...,"Rp 9,9 Miliar",5,6,300 m²,525 m²,2,HGB,4400 Watt,1,1,2,2,Baru,Luxury
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,DIJUAL RUMAH O RAYA MERR,Rp 10 Miliar,8,4,126 m²,200 m²,0,SHM,Lainnya,0,0,0,1,0,Luxury
281,DHARMAHUSADA INDAH,"Rp 7,9 Miliar",4,4,369 m²,550 m²,0,SHM,5500 Watt,1,0,0,1,0,Luxury
282,Dijual Rumah Modern Minimalis Bagus Siap Huni ...,"Rp 10,5 Miliar",6,3,450 m²,500 m²,2,SHM,Lainnya,0,1,2,2,Bagus,Luxury
283,Dijual Murah Rumah di Citraland Bukit Golf Sem...,"Rp 7,3 Miliar",4,4,375 m²,550 m²,1,SHM,10600 Watt,1,1,0,2,Bagus,Luxury
